In [3]:
!pip install --pre tensorflow
!pip install opencv-python==4.11.0.86 numpy==1.26.4 geopy==2.4.1 geocoder==1.38.1


  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached wrapt-1.17.2-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
  Using cached grpcio-1.73.1-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached numpy-2.1.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
  Using cach

In [4]:
import cv2
import numpy as np
import geocoder
from geopy.distance import geodesic
from tensorflow.keras.models import load_model
import os
import csv
from datetime import datetime

In [5]:
model = load_model('drone_classifier_no_mapping.h5')
class_names = ['Chinese Drone', 'Not a Drone', 'Turkish Drone']

In [6]:
# === Constants ===
KNOWN_WIDTH_CM = 150
FOCAL_LENGTH_PIXELS = 615
CONFIDENCE_THRESHOLD = 91
MIN_AREA_THRESHOLD = 500
ASPECT_RATIO_MIN = 0.5
ASPECT_RATIO_MAX = 2.0

In [7]:
os.makedirs("detections", exist_ok=True)
csv_file_path = "drone_detections.csv"

if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Label", "Confidence", "Distance_m", "Drone_Lat", "Drone_Lon", "Image_Path"])

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def get_current_location():
    g = geocoder.ip('me')
    if g.ok:
        return float(g.latlng[0]), float(g.latlng[1])
    return None, None

def estimate_drone_location(user_lat, user_lon, distance_m):
    origin = (user_lat, user_lon)
    drone_pos = geodesic(kilometers=distance_m / 1000).destination(origin, 0)
    return drone_pos.latitude, drone_pos.longitude

def generate_google_maps_url(user_lat, user_lon, drone_lat, drone_lon):
    return f"https://www.google.com/maps/dir/{user_lat},{user_lon}/{drone_lat},{drone_lon}"

def predict_from_frame(region):
    resized = cv2.resize(region, (224, 224))
    img_arr = resized.astype("float32") / 255.0
    img_arr = np.expand_dims(img_arr, axis=0)

    preds = model.predict(img_arr, verbose=0)
    class_idx = np.argmax(preds[0])
    label = class_names[class_idx]
    confidence = np.max(preds[0]) * 100
    return label, confidence


In [8]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera.")
    exit()

print(" Press 'q' to quit the live feed.")

user_lat, user_lon = get_current_location()
if user_lat is None:
    print("Unable to get current location.")
    cap.release()
    exit()

frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    height, width, _ = frame.shape
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    if len(faces) > 0:
        label = "Human Detected"
        confidence = 100.0
        distance_m = 0.0
        drone_lat, drone_lon = user_lat, user_lon
    else:
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            largest = max(contours, key=cv2.contourArea)
            area = cv2.contourArea(largest)
            if area > MIN_AREA_THRESHOLD:
                x, y, w, h = cv2.boundingRect(largest)
                aspect_ratio = w / h if h != 0 else 0

                if aspect_ratio < ASPECT_RATIO_MIN or aspect_ratio > ASPECT_RATIO_MAX:
                    label = "Ignored: Unusual Shape"
                    confidence = 0
                    distance_m = 0
                    drone_lat, drone_lon = user_lat, user_lon
                else:
                    roi = frame[y:y+h, x:x+w]
                    label, confidence = predict_from_frame(roi)

                    if confidence < CONFIDENCE_THRESHOLD:
                        label = "Ignored: Low Confidence"
                        distance_m = 0
                        drone_lat, drone_lon = user_lat, user_lon
                    else:
                        distance_cm = (KNOWN_WIDTH_CM * FOCAL_LENGTH_PIXELS) / max(w, 1)
                        distance_m = distance_cm / 100.0
                        drone_lat, drone_lon = estimate_drone_location(user_lat, user_lon, distance_m)

                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                        cv2.putText(frame, f"{label} ({confidence:.1f}%) | {distance_m:.1f}m",
                                    (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                        image_path = f"detections/frame_{timestamp}.jpg"
                        cv2.imwrite(image_path, frame)

                        with open(csv_file_path, mode='a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow([timestamp, label, f"{confidence:.2f}", f"{distance_m:.2f}",
                                             f"{drone_lat:.5f}", f"{drone_lon:.5f}", image_path])
            else:
                label = "No Drone Detected"
                confidence = 0
                distance_m = 0
                drone_lat, drone_lon = user_lat, user_lon
        else:
            label = "No Drone Detected"
            confidence = 0
            distance_m = 0
            drone_lat, drone_lon = user_lat, user_lon

    info = f"User: {user_lat:.4f}, {user_lon:.4f} | Drone: {drone_lat:.4f}, {drone_lon:.4f}"
    cv2.putText(frame, info, (10, height - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    cv2.imshow("Live Drone Classifier", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        if label not in ["Human Detected", "No Drone Detected", "Ignored: Low Confidence", "Ignored: Unusual Shape"]:
            print(f"\n Classification: {label} ({confidence:.2f}%)")
            print(f" Estimated Distance: {distance_m:.2f} meters")
            print(f" Drone Latitude: {drone_lat:.5f}, Longitude: {drone_lon:.5f}")
            print(f" Google Maps Link: {generate_google_maps_url(user_lat, user_lon, drone_lat, drone_lon)}")
        else:
            print(f"\n {label} — skipping drone prediction.")
        break

cap.release()
cv2.destroyAllWindows()

 Press 'q' to quit the live feed.

 Human Detected — skipping drone prediction.


In [1]:
import cv2
import numpy as np
import geocoder
from geopy.distance import geodesic
from tensorflow.keras.models import load_model
import os
import csv
from datetime import datetime
from norfair import Detection, Tracker

# === Load Drone Classifier Model ===
model = load_model('drone_classifier_no_mapping.h5')
class_names = ['Chinese Drone', 'Not a Drone', 'Turkish Drone']

# === Constants ===
KNOWN_WIDTH_CM = 150
FOCAL_LENGTH_PIXELS = 615
CONFIDENCE_THRESHOLD = 91
MIN_AREA_THRESHOLD = 500
ASPECT_RATIO_MIN = 0.5
ASPECT_RATIO_MAX = 2.0

# === Setup Folders and CSV ===
os.makedirs("detections", exist_ok=True)
csv_file_path = "drone_detections.csv"
if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Label", "Confidence", "Distance_m", "Drone_Lat", "Drone_Lon", "Image_Path"])

# === Location Utility Functions ===
def get_current_location():
    g = geocoder.ip('me')
    if g.ok:
        return float(g.latlng[0]), float(g.latlng[1])
    return None, None

def estimate_drone_location(user_lat, user_lon, distance_m):
    origin = (user_lat, user_lon)
    drone_pos = geodesic(kilometers=distance_m / 1000).destination(origin, 0)
    return drone_pos.latitude, drone_pos.longitude

def predict_from_frame(region):
    resized = cv2.resize(region, (224, 224))
    img_arr = resized.astype("float32") / 255.0
    img_arr = np.expand_dims(img_arr, axis=0)
    preds = model.predict(img_arr, verbose=0)
    class_idx = np.argmax(preds[0])
    label = class_names[class_idx]
    confidence = np.max(preds[0]) * 100
    return label, confidence

# === Norfair Tracker Setup ===
tracker = Tracker(
    distance_function=lambda det, trk: np.linalg.norm(det.points - trk.estimate),
    distance_threshold=30
)

# === Video Capture ===
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera.")
    exit()

print("Press 'q' to quit.")

user_lat, user_lon = get_current_location()
if user_lat is None:
    print("Unable to get current location.")
    cap.release()
    exit()

target_locked = False
target_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    height, width, _ = frame.shape

    detections = []

    if not target_locked:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > MIN_AREA_THRESHOLD:
                x, y, w, h = cv2.boundingRect(cnt)
                aspect_ratio = w / h if h != 0 else 0

                if ASPECT_RATIO_MIN <= aspect_ratio <= ASPECT_RATIO_MAX:
                    roi = frame[y:y + h, x:x + w]
                    label, confidence = predict_from_frame(roi)

                    if label != "Not a Drone" and confidence >= CONFIDENCE_THRESHOLD:
                        center_x = x + w // 2
                        center_y = y + h // 2
                        detections.append(Detection(points=np.array([[center_x, center_y]])))

                        distance_cm = (KNOWN_WIDTH_CM * FOCAL_LENGTH_PIXELS) / max(w, 1)
                        distance_m = distance_cm / 100.0
                        drone_lat, drone_lon = estimate_drone_location(user_lat, user_lon, distance_m)

                        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
                        image_path = f"detections/frame_{timestamp}.jpg"
                        cv2.imwrite(image_path, frame)

                        with open(csv_file_path, mode='a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow([timestamp, label, f"{confidence:.2f}", f"{distance_m:.2f}",
                                             f"{drone_lat:.5f}", f"{drone_lon:.5f}", image_path])

                        target_locked = True
                        target_id += 1
                        print(f"Target locked: {label} at {distance_m:.1f}m")
                        break  # Lock onto the first valid target

    # Tracker Update
    tracked_objects = tracker.update(detections)

    for obj in tracked_objects:
        pos = obj.estimate[0]
        cv2.circle(frame, (int(pos[0]), int(pos[1])), 5, (255, 0, 0), -1)
        cv2.putText(frame, f"Target #{target_id}", (int(pos[0]), int(pos[1]) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    info = f"User: {user_lat:.4f}, {user_lon:.4f} | Target Locked: {'Yes' if target_locked else 'No'} | Target ID: {target_id if target_locked else '-'}"
    cv2.putText(frame, info, (10, height - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

    cv2.imshow("Drone Classifier with Norfair Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Press 'q' to quit.
Target locked: Chinese Drone at 4.4m


In [2]:
!pip show opencv-python
!pip show numpy
!pip show geocoder
!pip show geopy
!pip show tensorflow
!pip show norfair


Name: opencv-python
Version: 4.11.0.86
Summary: Wrapper package for OpenCV python bindings.
Home-page: https://github.com/opencv/opencv-python
Author: 
Author-email: 
License: Apache 2.0
Location: C:\Users\rohan\anaconda3\envs\drone-env\Lib\site-packages
Requires: numpy
Required-by: 
Name: numpy
Version: 1.26.4
Summary: Fundamental package for array computing in Python
Home-page: https://numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: Copyright (c) 2005-2023, NumPy Developers.
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are
met:

    * Redistributions of source code must retain the above copyright
       notice, this list of conditions and the following disclaimer.

    * Redistributions in binary form must reproduce the above
       copyright notice, this list of conditions and the following
       disclaimer in the documentation and/or other materials pr